In [ ]:
### imports

# external modules
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import importlib

# local modules
sys.path.append('../utils')
import csv_utils as csvu
import json_utils as jsonu
import dataframe_utils as dfu
import hist_utils as hu
import autoencoder_utils as aeu
import plot_utils as pu
import generate_data_utils as gdu
import refruns_utils as rru
importlib.reload(csvu)
importlib.reload(jsonu)
importlib.reload(dfu)
importlib.reload(hu)
importlib.reload(aeu)
importlib.reload(pu)
importlib.reload(gdu)
importlib.reload(rru)
sys.path.append('../src')
sys.path.append('../src/classifiers')
import PCAClassifier
importlib.reload(PCAClassifier)

In [ ]:
### load test histograms
    
histname = 'size_PXLayer_3'
dffile = '../data/DF2017_{}.csv'.format(histname)
histdf = csvu.read_csv(dffile)
histdf = dfu.select_dcson(histdf)
histdf = dfu.select_highstat(histdf)
print(len(histdf))

(hists_train) = hu.preparedatafromdf(histdf, donormalize=True, doplot=False)
print(hists_train.shape)

In [ ]:
### fit a PCA model

classifier = PCAClassifier.PCAClassifier( ncomponents=0, nmax=10 )
classifier.train( hists_train )

In [ ]:
### dispaly the PCA components

comps = classifier.get_components()
print(comps.shape)
if comps.shape[0]>0: pu.plot_hists_multi(comps, title=histname, colorlist=list(range(len(comps))) )

In [ ]:
### display some example reconstructions

inds = [0]
for ind in inds:
    
    # automatically
    hist = hists_train[ind:ind+1,:]
    reco = classifier.reconstruct(hist)
    pu.plot_hists( [hist[0],reco[0]], 
                    colorlist=['black','red'], 
                    labellist=['orig','reco'] )
    
    # manually
    hist = hists_train[ind:ind+1,:]
    red = classifier.PCA.transform(hist)
    print(red)
    reco = classifier.PCA.inverse_transform(red)
    pu.plot_hists( [hist[0],reco[0]], 
                    colorlist=['black','red'], 
                    labellist=['orig','reco'] )
    
    
    # even more manually
    hist = hists_train[ind:ind+1,:]
    red = np.dot(classifier.PCA.components_, np.transpose(hist - classifier.PCA.mean_))
    if red.shape[0]==0: reco = [classifier.PCA.mean_] # special for 0 components
    else: reco = np.dot(red, classifier.PCA.components_) + classifier.PCA.mean_
    pu.plot_hists( [hist[0],reco[0]], 
                    colorlist=['black','red'], 
                    labellist=['orig','reco'] )